# MoReBo
MoReBo stands for Movie Recommendations Bot.

In [1]:
import pandas as pd
import numpy as np

# Read the data
movies = pd.read_csv('data/movies.csv')
ratings = pd.read_csv('data/ratings.csv')
tags = pd.read_csv('data/tags.csv')
links = pd.read_csv('data/links.csv')

# Merge the data
merged = pd.merge(movies, ratings, on='movieId', how='inner')
merged = pd.merge(merged, tags, on='movieId', how='inner')
merged = pd.merge(merged, links, on='movieId', how='inner')

print(merged)

TypeError: merge() got multiple values for argument 'on'